In [ ]:
%matplotlib inline
from pyvista import set_plot_theme
set_plot_theme("document")


# Observations covariances, Localisation matrix

The notebook illustrate how to read EM sensors dataset to be prepare for DA

*Estimated time to run the notebook = 2min*


In [ ]:
import numpy as np
from pyCATHY.DA.cathy_DA import DA
import pandas as pd
import matplotlib.pyplot as plt
from pyCATHY.DA.cathy_DA import DA, dictObs_2pd
from pyCATHY.DA.observations import read_observations, prepare_observations, make_data_cov
from pyCATHY.DA.cathy_DA import DA, dictObs_2pd, localisation
from pathlib import Path
import pyvista as pv
import os

-----------------------



In [ ]:
rootpath = Path(os.getcwd()).parent

# Create a CATHY project
# -----------------------
simuWithDA = DA(
                        dirName='.',
                        #prj_name= choose_simulation.value,
                        prj_name= 'test_covariance_localisation',
                        notebook=True,
                    )

simuWithDA.run_preprocessor()
simuWithDA.run_processor(IPRT1=3)

In [ ]:
dict_obs = {} # initiate the dictionnary

# ERT observations metadata
# -------------------------
metadata_EM   = {
                    'data_type': '$EM$', # units
                    'units': '$\mS/m$', # units transfer_resistances
                    'instrument': 'GF6L', # Instrument
                    'data_format': 'emagpy', # format (raw or preprocessed)
                    'fwdEMnoise': 5, #
                }

# sw_times_common_rounded.insert(0,0)
date_string0 = '2023-01-01 08:00:00.00000'
start_date = pd.to_datetime(date_string0)
datetime_EM = pd.date_range(start=start_date, periods=60, freq='D')  # 60 daily points
elapsed_seconds = (datetime_EM - start_date).total_seconds()


for i in selected_indices:
    tt = elapsed_seconds[i]
    # .txt file observation generated by emagpy
    filename = fwdEM_path / f'{args.SolfwdName}/fwd_ECa{i}.txt'

    data_measure = read_observations(
                                    dict_obs,
                                    filename,
                                    data_type = 'EM',
                                    data_err = args.dataErr, # instrumental error
                                    show=True,
                                    tA=tt,
                                    obs_cov_type='data_err', #data_err
                                    meta=metadata_EM,
                                    datetime=datetime_EM[i],
                                    )

data_measure_df = dictObs_2pd(dict_obs)
data_measure_df.head()
# data_measure_df.columns
# dict_obs.keys()
data_measure_df.iloc[0]['data']
# data_measure_df.index

In [ ]:
nnod = simuWithDA.grid3d['nnod']
grid_coords = simuWithDA.grid3d['mesh3d_nodes'][0:int(nnod),:2]
# Example usage
ncoils =  6
L = 4
# Build localization matrix (coils independent)
local_mat = localisation.build_localization_matrix(grid_coords, 
                                                   ncoils,
                                                   L, 
                                                   with_coil_covariance=True
                                                   )

# Visualize a small block
plt.figure(figsize=(6, 6))
plt.imshow(local_mat[:60, :60], cmap='viridis')
plt.colorbar(label='Localization weight')
plt.title('Localization for Coils at the Same Grid Point')
print("Localization matrix shape:", local_mat.shape)
simuWithDA.localisationMatrix = [local_mat]*len(data_measure_df.index)
# np.shape([local_mat]*len(data_measure_df.index))